You will need a Google Cloud service account with permission to write the bucket. That account will be assocated with a project.

In [ ]:
from siphon.catalog import TDSCatalog
from pathlib import Path
from google.cloud import storage
import io, os
import urllib.request
from urllib.parse import urlparse
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
load_dotenv(os.path.expanduser('~/.cefi_env'))

In [ ]:
bucket_name = 'noaa-oar-cefi-regional-mom6'
project_id = os.getenv("GOOGLE_PROJECT_ID")
storage_client = storage.Client.from_service_account_json(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
match = 'cefi_portal'
catalog_root = 'https://psl.noaa.gov/thredds/catalog/Projects/CEFI/regional_mom6/cefi_portal/northeast_pacific/full_domain/catalog.xml'

catalog_prefix_index = catalog_root.find(match)
catalog_replace = catalog_root[:catalog_prefix_index + len(match) + 1]
print(catalog_replace)

In [ ]:
def get_datasets(root, total):
    if root.datasets and 'latest' not in root.catalog_url and 'r20240213' not in root.catalog_url:
        downloads = 0
        for dataset in root.datasets:
            access_url = root.datasets[dataset].access_urls['HTTPServer']
            dataset_prefix = access_url.find(match)
            url_replace = access_url[:dataset_prefix + len(match) + 1]
            outfile = access_url.replace(url_replace,'')
            downloads = downloads  + 1
            file_path = Path(outfile)
            file_path.parent.mkdir(parents=True, exist_ok=True)
            destination_blob_name = outfile
            blob = bucket.blob(destination_blob_name)
            if not blob.exists():
                urllib.request.urlretrieve(access_url, outfile)
                blob.upload_from_filename(outfile, content_type='application/x-netcdf')
                os.remove(outfile)
        if downloads > 0:
            total = total + downloads
            print(root.catalog_url)
            print(f'Downloaded {downloads} files.')
    for catalog in root.catalog_refs:
        child = root.catalog_refs[catalog].follow()
        total = get_datasets(child, total)
    return total

In [ ]:
cat = TDSCatalog(catalog_root)
total = 0
total = get_datasets(cat, total)
print(f'Total number of files transferred = {total}')